#Install dependencies and import libraries

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.4 MB/s eta 0:00:00


In [ ]:
# pip install xformers

In [2]:
import pandas as pd
from transformers import pipeline
import csv

# Data Load

In [3]:
ClickBaitdataset = pd.read_csv('/content/clickbait_data.csv')

#Load the desired model from Hugging face


> Model trained from roberta-base on the go_emotions dataset for multi-label classification.
go_emotions is based on Reddit data and has 28 labels. It is a multi-label dataset where one or multiple labels may apply for any given input text, hence this model is a multi-label classification model with 28 'probability' float outputs for any given input text. Typically a threshold of 0.5 is applied to the probabilities for the prediction for each label.


resource: https://huggingface.co/SamLowe/roberta-base-go_emotions?text=Obama+Welcomes+Specter+to+the+Party





#Emotions for the clickbait headlines

In [4]:
#only the clickbaites observations  
CBheadlines = ClickBaitdataset[ClickBaitdataset['clickbait'] == 1]

#only the headline column of the clickbait observations
CBheadlines = CBheadlines['headline']

In [6]:
# Using a specific model for sentiment analysis on the clickbait headlines 
specific_model = pipeline(model="SamLowe/roberta-base-go_emotions")
sa_clickbaite = pd.DataFrame.from_dict(specific_model(CBheadlines.tolist()))

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
sa_clickbaite.label.value_counts()

neutral           11500
curiosity          1452
admiration          973
amusement           462
excitement          202
love                159
surprise            136
annoyance           123
sadness             122
disgust             118
joy                 117
approval            106
fear                 93
disappointment       80
anger                61
confusion            59
embarrassment        48
caring               47
disapproval          33
desire               29
realization          27
optimism             21
gratitude            19
nervousness           6
remorse               6
Name: label, dtype: int64

In [10]:
#to observe the labels which are the emotions over the headlines 
sa_clickbaite.label

0           neutral
1         curiosity
2        excitement
3        admiration
4           neutral
            ...    
15994       neutral
15995       neutral
15996       neutral
15997       neutral
15998    admiration
Name: label, Length: 15999, dtype: object

In [11]:
#will add a new column (emotions) to the dataset 
#which will include the emotions of each clickbait headline
ClickBaitdataset['emotions'] = sa_clickbaite['label']

In [14]:
#to have an overview of the dataset with the new column 
ClickBaitdataset.head()

,headline,clickbait,emotions
0,Should I Get Bings,1,neutral
1,Which TV Female Friend Group Do You Belong In,1,curiosity
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1,excitement
3,"This Vine Of New York On ""Celebrity Big Brothe...",1,admiration
4,A Couple Did A Stunning Photo Shoot With Their...,1,neutral


In [76]:
ClickBaitdataset = ClickBaitdataset[ClickBaitdataset['clickbait'] == 1]

In [77]:
#a new csv file that contain the clickbait headlines with the emotion column 
ClickBaitdataset.to_csv('clickbaitE.csv')

#Emotions for the non-Clickbait

In [19]:
#the non-clickbait observations  
NNCBheadlines = ClickBaitdataset[ClickBaitdataset['clickbait'] == 0]

#only the headline column for the non-clickbait observations 
NNCBheadlines = NNCBheadlines['headline']

In [21]:
# Using a specific model for sentiment analysis on the non-clickbait headlines 
specific_model = pipeline(model="SamLowe/roberta-base-go_emotions")
sa_nonClickbait = pd.DataFrame.from_dict(specific_model(NNCBheadlines.tolist()))

In [23]:
#count the frequency of each emotion in relation with the headline in the non-clickbait examples 
sa_nonClickbait.label.value_counts()

neutral           13945
sadness            1317
disappointment      288
disapproval          61
fear                 58
admiration           53
curiosity            43
excitement           40
confusion            28
remorse              26
approval             25
surprise             22
optimism             17
nervousness          16
joy                  14
disgust              10
amusement             6
annoyance             6
caring                6
gratitude             6
embarrassment         5
anger                 4
desire                3
love                  1
realization           1
Name: label, dtype: int64

In [24]:
#obtain the non clickbait headlines
NCB = ClickBaitdataset[ClickBaitdataset['clickbait'] == 0]

In [26]:
#take out of the non clickbait headline the "headline" column only 
NCBheadlines = NCB['headline']

In [28]:
# a csv file that containes the non clickbait headlines only 
NCB.to_csv('nonClickBE.csv')

In [36]:
#read the csv file that contain the non clickbait headlines only 
NCB = pd.read_csv('/content/nonClickBE.csv')
NCB = NCB.drop(columns=['Unnamed: 0'])

In [38]:
#include the emotions column to the new csv file which containes the non clickbait headlines 
NCB['emotions'] = sa_nonClickbait['label']

In [39]:
#show the dataset (with non clickbait headline only) after adding the emotions column 
NCB.head()

,headline,clickbait,emotions
0,Bill Changing Credit Card Rules Is Sent to Oba...,0,neutral
1,"In Hollywood, the Easy-Money Generation Toughe...",0,neutral
2,1700 runners still unaccounted for in UK's Lak...,0,sadness
3,Yankees Pitchers Trade Fielding Drills for Put...,0,neutral
4,Large earthquake rattles Indonesia; Seventh in...,0,neutral


In [40]:
#save a csv file that containes the non clickbait headlines only 
NCB.to_csv('nonClickBwithE.csv')

#Merge the two csv files which containes all of the headlines with the emotion column 

In [78]:
ClickBaitE = pd.read_csv('/content/clickbaitE.csv')
ClickBaitE = ClickBaitE.drop(columns=['Unnamed: 0'])

In [80]:
nonClickBaitE = pd.read_csv('/content/nonClickBwithE.csv')
nonClickBaitE = nonClickBaitE.drop(columns=['Unnamed: 0'])

In [82]:
clickBaitDatasetWithE = pd.merge(ClickBaitE, nonClickBaitE, how='outer')

In [83]:
clickBaitDatasetWithE

,headline,clickbait,emotions
0,Should I Get Bings,1,neutral
1,Which TV Female Friend Group Do You Belong In,1,curiosity
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1,excitement
3,"This Vine Of New York On ""Celebrity Big Brothe...",1,admiration
4,A Couple Did A Stunning Photo Shoot With Their...,1,neutral
...,...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0,neutral
31996,"British Liberal Democrat Patsy Calton, 56, die...",0,sadness
31997,Drone smartphone app to help heart attack vict...,0,neutral
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0,neutral


In [91]:
clickBaitDatasetWithE.to_csv('clickBaitDatasetWithE.csv')